In [ ]:
!pip install netCDF4
!pip3 install numpy pylibtiff
!pip install georasters

In [ ]:
from google.colab import drive
drive.mount('/content/drive') #"/content/drive/MyDrive/forests/ESACCI-LC-L4-LCCS-Map-300m-P1Y-"+str(i)+"-v2.0.7cds.nc"

In [ ]:
import matplotlib.pyplot as plt
import netCDF4
import numpy as np
import string

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid

import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plot
%matplotlib inline



import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)


from libtiff import TIFF
import georasters as gr
import xarray as xr

In [ ]:
# installations ( uncomment and run this cell ONLY when using google colab or kaggle )

!pip install s3fs --quiet

# # properly install cartopy in colab to avoid session crash
!apt-get install libproj-dev proj-data proj-bin --quiet
!apt-get install libgeos-dev --quiet
!pip install cython --quiet
!pip install cartopy --quiet
!pip install geoviews

!apt-get -qq install python-cartopy python3-cartopy  --quiet
!pip uninstall -y shapely  --quiet
!pip install shapely --no-binary shapely  --quiet

!pip install boto3 --quiet

# you may need to restart the runtime after running this cell and that is ok

In [ ]:
import s3fs
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy
import cartopy.crs as ccrs
import boto3
import botocore
import os
import pooch
import tempfile
import holoviews
from geoviews import Dataset as gvDataset
import geoviews.feature as gf
from geoviews import Image as gvImage

In [ ]:
dataset = netCDF4.Dataset('/content/drive/MyDrive/forests/ESACCI-LC-L4-LCCS-Map-300m-P1Y-1992-v2.0.7cds.nc', 'r')
variable_names = list(dataset.variables)
print(variable_names)

In [ ]:
land_cover=dataset.variables['lccs_class']
print(land_cover)

In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Load the dataset using xarray with Dask for lazy loading
ds = xr.open_dataset('/content/drive/MyDrive/forests/ESACCI-LC-L4-LCCS-Map-300m-P1Y-1992-v2.0.7cds.nc', chunks={'time': 1})

# Access the lccs_class variable as a DataArray
land_cover = ds['lccs_class'].sel(time="1992-01-01")  # Select the desired time slice

# Optionally downsample the data if it's too large
land_cover_downsampled = land_cover.coarsen(lat=2, lon=2).mean()  # Downsample by averaging

# Define the color map based on the flag_colors provided
flag_colors = [
    "#ffff64", "#ffff64", "#ffff00", "#aaf0f0", "#dcf064", "#c8c864",
    "#006400", "#00a000", "#00a000", "#aac800", "#003c00", "#003c00",
    "#005000", "#285000", "#285000", "#286400", "#788200", "#8ca000",
    "#be9600", "#966400", "#966400", "#966400", "#ffb432", "#ffdcd2",
    "#ffebaf", "#ffc864", "#ffd278", "#ffebaf", "#00785a", "#009678",
    "#00dc82", "#c31400", "#fff5d7", "#dcdcdc", "#fff5d7", "#0046c8",
    "#ffffff"
]

# Create a colormap
cmap = mcolors.ListedColormap(flag_colors)

# Create a figure and axis
plt.figure(figsize=(12, 8))
plt.imshow(land_cover_downsampled, cmap=cmap, interpolation='nearest', vmin=0, vmax=220)
plt.colorbar(ticks=np.arange(0, 221, 10), label='LCCS Class')
plt.title('Land Cover Classification (LCCS Class) for 1992')
plt.xlabel('Longitude Index')
plt.ylabel('Latitude Index')
plt.grid(False)  # Disable grid for better visualization
plt.show()

# Clear variables to free memory
del ds
import gc
gc.collect()  # Call garbage collector